In [1]:
### Author: Pongpisit Thanasutives ###
%load_ext autoreload
%autoreload 2

import os
from itertools import combinations
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, Normalizer
from tqdm import trange
from scipy import io as sio
from scipy.stats import uniform, norm
import pysindy as ps
import pocomc as pc
from bayesian_model_evidence import log_evidence

In [2]:
### algorithm 2 ###
def TopRsq(X_full, y, m, n_tops=25):
    n_feats = X_full.shape[-1]
    r_scores = []
    models = []
    for comb in combinations(range(n_feats), m):
        comb = list(comb)
        active_indices = np.zeros(n_feats)
        active_indices[comb] = 1
        X_sub = X_full[:, comb]
        lr = LinearRegression(fit_intercept=False).fit(X_sub, y)
        R2 = lr.score(X_sub, y)
        r_scores.append(R2)
        models.append(active_indices)
    r_scores = np.array(r_scores)
    r_argsort = np.argsort(r_scores)[::-1][:n_tops]
    r_scores = r_scores[r_argsort]
    models = np.array(models)
    models = models[r_argsort].T
    rating = np.dot(models, r_scores)
    return models, r_scores, rating

In [3]:
### algorithm 3 ###
def comprehensive_search(X_full, y, max_support_size=8, n_tops=None, threshold=0.75):
    X = X_full.copy()
    n_feats = X_full.shape[-1]
    n_tops = int(np.ceil(n_feats/2)) if n_tops is None else n_tops
    ratings = np.zeros((n_feats, max_support_size))
    search = True; support_size = 1
    optimal_indices = None
    i0 = np.arange(n_feats)
    while search and support_size <= max_support_size:
        _, _, rating = TopRsq(X, y, m=support_size, n_tops=n_tops)
        rating = rating/rating.max()
        ratings[:, support_size-1][i0] = rating
        if support_size >= 2:
            i0 = np.nonzero(ratings[:, support_size-1] + ratings[:, support_size-2])[0]
            X = X_full[:, i0]
            i1 = np.where(ratings[:, support_size-1] > threshold)[0]
            i2 = np.where(ratings[:, support_size-2] > threshold)[0]
            if len(i1) == len(i2) and np.all(i1 == i2):
                search = False
                optimal_indices = i1
        support_size += 1
    return optimal_indices, ratings[:, :support_size-1]

In [4]:
n_experiments = 100
n_samples = 10000
n_features = 8
n_informative = 2

threshold = 0.25
max_support_size = 8

success = 0
for i in trange(n_experiments):
    X_train, y_train = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative)
    X_train = StandardScaler().fit_transform(X_train)
    y_train = StandardScaler().fit_transform(y_train.reshape(-1, 1))
    top_models, _, _ = TopRsq(X_train, y_train, m=n_informative)
    true_indices = np.where(top_models[:, 0] > 0)[0]
    est_indices, ratings = comprehensive_search(X_train, y_train, max_support_size=max_support_size, threshold=threshold)
    if est_indices is not None and len(true_indices) == len(est_indices) and np.all(true_indices == est_indices):
        success += 1
        
success/n_experiments

100%|████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.29it/s]


0.89

### PDE ###

In [5]:
data_path = "./Datasets/"
data = sio.loadmat(os.path.join(data_path, "KdV_rudy.mat"))
u_clean = (data['usol']).real; u = u_clean.copy()
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]; dx = x[2]-x[1]

np.random.seed(0)
noise_type = "gaussian"
noise_lv = float(50)
print("Noise level:", noise_lv)
noise = 0.01*np.abs(noise_lv)*(u.std())*np.random.randn(u.shape[0],u.shape[1])
u = u + noise
u = np.load("./Denoised_data/kdv_gaussian50_bm3d.npy")

xt = np.array([x.reshape(-1, 1), t.reshape(1, -1)], dtype=object)
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

Noise level: 50.0


In [6]:
function_library = ps.PolynomialLibrary(degree=2, include_bias=False)

weak_lib = ps.WeakPDELibrary(
    function_library=function_library,
    derivative_order=3,
    spatiotemporal_grid=XT,
    include_bias=True,
    diff_kwargs={"is_uniform":True},
    K=10000
)

X_pre = np.array(weak_lib.fit_transform(np.expand_dims(u, -1)))
y_pre = weak_lib.convert_u_dot_integral(np.expand_dims(u, -1))
y_stan = StandardScaler().fit_transform(y_pre)
N = len(y_pre)

In [7]:
effective_indices, rating = comprehensive_search(X_pre, y_stan, 
                                                 max_support_size=max_support_size,  
                                                 threshold=0.75)
effective_indices, rating

(array([5, 6]),
 array([[0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [1.        , 1.        , 0.59792465, 0.50007196],
        [0.        , 0.        , 0.        , 0.        ],
        [0.57037676, 0.25715582, 1.        , 1.        ],
        [0.96889968, 0.75781667, 0.80035118, 1.        ],
        [0.88294231, 0.25069132, 0.59787182, 0.49998517],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.78206225, 0.49947882, 0.39822583, 0.49997189],
        [0.76565043, 0.24996776, 0.20007144, 0.49997098]]))

In [8]:
np.nonzero(TopRsq(X_pre, y_pre, m=1)[0][:, 0])[0], \
np.nonzero(TopRsq(X_pre, y_pre, m=2)[0][:, 0])[0], \
np.nonzero(TopRsq(X_pre, y_pre, m=3)[0][:, 0])[0]

(array([3]), array([5, 6]), array([3, 5, 6]))

In [9]:
from exhaustive_selection import best_subset
from functools import partial
from p_tqdm import p_map

In [10]:
best_subset_results = p_map(partial(best_subset, X_pre, y_pre), [1, 2, 3, 4, 5])
best_subset_results

  0%|          | 0/5 [00:00<?, ?it/s]

[(3,), (5, 6), (3, 5, 6), (3, 5, 6, 7), (3, 5, 6, 7, 10)]

### Exat model evidence ###

In [11]:
bme = [log_evidence(X_pre, y_pre, effective_indices, v=1e-2, standardize=False) for effective_indices in best_subset_results]
bme, np.argmax(bme)+1

([5791.343919431005,
  5845.8666839311,
  5843.473219566753,
  5843.443861699447,
  5842.85127357445],
 2)

### Bayes factor ###

In [12]:
effective_indices = [5, 6]
X_pre_sub = X_pre[:, effective_indices].copy().T
def log_likelihood(param):
    global X_pre_sub, y_pre, N
    ssr = np.sum(np.abs(param@X_pre_sub - y_pre.flatten())**2, axis=-1)
    def ssr2llf(ssr, nobs):
        nobs2 = nobs / 2.0
        llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
        return llf
    return ssr2llf(ssr, N)

n_dim = len(effective_indices)
prior = pc.Prior(n_dim*[norm(0, 1)])
sampler = pc.Sampler(
    prior=prior,
    likelihood=log_likelihood,
    vectorize=True,
)
sampler.run()
simple_samples, simple_weights, _, _  = sampler.posterior()
logz_simple, logz_err_simple = sampler.evidence()

/Users/pongpisitthanasutives/Desktop/research/pocomc/pocomc/mcmc.py:132: RuntimeWarning: overflow encountered in exp559]    
  np.exp(logl_prime * beta - logl * beta + logp_prime - logp + logdetj_prime - logdetj + logdetj_flow_prime - logdetj_flow - A + B)
Iter: 49it [00:51,  1.05s/it, beta=1, calls=111104, ESS=3841, logZ=3.19e+4, logP=3.19e+4, acc=0.352, steps=10, eff=0.588]


In [13]:
effective_indices = [3, 5, 6]
X_pre_sub = X_pre[:, effective_indices].copy().T
def log_likelihood(param):
    global X_pre_sub, y_pre, N
    ssr = np.sum(np.abs(param@X_pre_sub - y_pre.flatten())**2, axis=-1)
    def ssr2llf(ssr, nobs):
        nobs2 = nobs / 2.0
        llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
        return llf
    return ssr2llf(ssr, N)

n_dim = len(effective_indices)
prior = pc.Prior(n_dim*[norm(0, 1)])
sampler = pc.Sampler(
    prior=prior,
    likelihood=log_likelihood,
    vectorize=True,
)
sampler.run()
extended_samples, extended_weights, _, _  = sampler.posterior()
logz_extended, logz_err_extended = sampler.evidence()

Iter: 59it [01:11,  1.21s/it, beta=1, calls=114944, ESS=3875, logZ=3.52e+4, logP=3.53e+4, acc=0.164, steps=10, eff=0.699]  


In [14]:
# Bayes factor of extended to simple model
BF = np.exp(logz_extended-logz_simple)
if BF > 1:
    print('The extended model is more probable than the simple model.')
else:
    print('The simple model is more probable than the extended model.')

The extended model is more probable than the simple model.


/var/folders/93/k1wfjtks3ln5whhs7nszdspw0000gn/T/ipykernel_1472/862312044.py:2: RuntimeWarning: overflow encountered in exp
  BF = np.exp(logz_extended-logz_simple)


In [15]:
np.dot(simple_weights, simple_samples)

array([-0.6624994 , -5.11376209])

In [16]:
np.dot(extended_weights, extended_samples)

array([-0.15187746, -0.80474109, -4.96284739])